<a href="https://www.kaggle.com/code/jpthirumalai/pharmacovigilance-ver1-hcls?scriptVersionId=234498267" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"""
Capstone Project: Multi-Agent System for Real-Time Pharmacovigilance Signal Detection<br>
Date: 2025-04-08
"""

In [2]:


# %% [markdown]
# # Capstone: Real-Time Pharmacovigilance Signal Detection Agent System
#
# **Goal:** Monitor diverse data sources (literature, news, simulated regulatory/social feeds) to identify potential drug safety signals using a multi-agent system powered by Gemini and Vector Search.
#
# **Core Components:**
# 1.  **Data Ingestion:** Fetch/Simulate data from PubMed, News, Social Media, FAERS.
# 2.  **Embedding & Vector Store:** Embed relevant text data and store in ChromaDB.
# 3.  **Knowledge Store:** Basic info on drugs, known ADRs (simulated).
# 4.  **Generative AI Agents:** Specialized agents for scanning, context analysis, and synthesis.
# 5.  **Agent Communication:** Simple function calls or message passing.
# 6.  **LLM:** Google Gemini (`gemini-1.5-flash-latest` or `gemini-1.5-pro-latest`).


In [3]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:00

In [4]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=9ba8a01dfcd6cb1171c0fa78d3566b15ea2a103e4ebdc598f44ce9c2f35eedb5
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [5]:
from google import genai
from google.genai import types
import google.generativeai as ogenai

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [6]:
"""
Capstone Project: Multi-Agent System for Real-Time Pharmacovigilance Signal Detection
Date: 2025-04-08
"""

# %% [markdown]
# # Capstone: Real-Time Pharmacovigilance Signal Detection Agent System
#
# **Goal:** Monitor diverse data sources (literature, news, simulated regulatory/social feeds) to identify potential drug safety signals using a multi-agent system powered by Gemini and Vector Search.
#
# **Core Components:**
# 1.  **Data Ingestion:** Fetch/Simulate data from PubMed, News, Social Media, FAERS.
# 2.  **Embedding & Vector Store:** Embed relevant text data and store in ChromaDB.
# 3.  **Knowledge Store:** Basic info on drugs, known ADRs (simulated).
# 4.  **Generative AI Agents:** Specialized agents for scanning, context analysis, and synthesis.
# 5.  **Agent Communication:** Simple function calls or message passing.
# 6.  **LLM:** Google Gemini (`gemini-1.5-flash-latest` or `gemini-1.5-pro-latest`).

# %%
# # 1. Setup: Libraries and API Keys
import os
import json
import re
import datetime
import time
import hashlib # For generating consistent IDs

# Core AI/VectorStore Libs
# import google.generativeai as genai
import chromadb
from chromadb.utils import embedding_functions

# Data Source Libs (Install as needed)
import requests # For NewsAPI, other web APIs
# from Bio import Entrez # For PubMed - Requires setup
import feedparser # For RSS Feeds (News, some journals)
# import praw # For Reddit - Requires API setup

print(f"Notebook started on: {datetime.datetime.now()}")
print(f"Current date context: Tuesday, April 8, 2025") # As per user context


Notebook started on: 2025-04-17 16:33:45.313913
Current date context: Tuesday, April 8, 2025


In [7]:
# --- Securely load API keys ---
# Recommended: Use environment variables or Colab secrets
# Example:
# GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
# NEWS_API_KEY = os.environ.get("NEWS_API_KEY")
# REDDIT_CLIENT_ID = os.environ.get("REDDIT_CLIENT_ID")
# etc.

In [8]:
from kaggle_secrets import UserSecretsClient

google_client = UserSecretsClient().get_secret("GOOGLE_API_KEY")
news_client = UserSecretsClient().get_secret("NEWS_API_KEY")

2. Initialize LLM & Embedding Model (Using Google AI)

In [9]:
generation_config =[ 
    types.GenerateContentConfig(
      temperature = 0.7, # Adjust for creativity vs consistency
      top_p= 1.0,
      top_k= 32,
      max_output_tokens= 8192,
    )
]


In [10]:
# Safety settings for Gemini
# safety_settings = [
#     {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
#     {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
#     {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
#     {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
# ]

safety_settings=[
        types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        ),]

In [11]:
# import google.generativeai as genai2
client = None
try:
    client = genai.Client(api_key=google_client)
    # Using Flash for speed, consider Pro for more complex reasoning
    model_name="gemini-2.0-flash-001",
    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents="Will this pfizer or abbvie solve any of their use cases with Pharmacovigilance",
        # config = generation_config,
        config = types.GenerateContentConfig(
            temperature = 0.7, # Adjust for creativity vs consistency
            top_p= 1.0,
            top_k= 32,
            max_output_tokens= 8192,
            safety_settings=[
            types.SafetySetting(
                category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
            ),]
        )
        # safetySettings = safety_settings
    )
    print(response.text)
    print(f"Gemini model '{model_name}' initialized.")

    # Using Google's text embedding model via the Generative AI SDK
    embedding_model_name = "models/text-embedding-004"
    # Note: Direct embedding function via genai SDK might be simpler for some use cases
    # Or use ChromaDB's helper with GoogleGenerativeAiEmbeddingFunction if needed
    google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=google_client, model_name=embedding_model_name)
    print(f"Google Embedding model '{embedding_model_name}' ready via ChromaDB helper.")

except Exception as e:
    print(f"Error initializing Google AI services: {e}")
    # Handle error appropriately (e.g., exit or fallback)


Let's break down how Pfizer and AbbVie might leverage AI solutions for pharmacovigilance and consider potential use cases:

**Understanding the Players and the Challenge: Pharmacovigilance**

*   **Pharmacovigilance (PV):**  The science and activities relating to the detection, assessment, understanding, and prevention of adverse effects or any other drug-related problem.  It's a critical function for pharmaceutical companies to ensure patient safety and maintain regulatory compliance.  It involves collecting reports of adverse events (side effects) associated with their drugs, analyzing the data, and taking action if necessary (e.g., updating labeling, issuing warnings, or even withdrawing a drug from the market).
*   **Key Challenges in Pharmacovigilance:**
    *   **Volume of Data:**  Massive amounts of data come from various sources: clinical trials, spontaneous reports from patients and healthcare professionals, social media, scientific literature, and more.
    *   **Data Heterog

### Helper function for LLM calls

In [12]:
def call_gemini(prompt, llm_model=client, is_json_output=False):
    """Sends a prompt to the Gemini model and returns the text response."""
    try:
        client = genai.Client(api_key=google_client)
        # model_name="gemini-2.0-flash-001",
        # Add instruction for JSON output if requested
        if is_json_output:
             prompt += "\n\nPlease format your response strictly as a JSON object."

        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = prompt,
            # config=generation_config,
            # safetySettings=safety_settings
            config = types.GenerateContentConfig(
                temperature = 0.7, # Adjust for creativity vs consistency
                top_p= 1.0,
                top_k= 32,
                max_output_tokens= 8192,
                safety_settings=[
                types.SafetySetting(
                    category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                    threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                )]
            )
        )
        # Basic check for blocked content
        if not response.candidates:
             print("Warning: Response was blocked or empty.")
             return None
        # Attempt to extract text, handle potential errors
        try:
            result_text = response.text
            if is_json_output:
                # Clean potential markdown ```json ... ```
                result_text = re.sub(r'^```json\s*|\s*```$', '', result_text, flags=re.MULTILINE)
                return json.loads(result_text) # Parse JSON
            return result_text
        except (ValueError, json.JSONDecodeError) as json_err:
             print(f"Warning: Could not parse expected JSON output. Error: {json_err}")
             print(f"Raw response: {response.text}")
             return None # Or return raw text if preferred fallback
        except Exception as resp_err:
            print(f"Warning: Error extracting text from response. Error: {resp_err}")
            return None

    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        print(traceback.format_exc())
               
        return None

### Helper function for Embeddings

In [13]:
def get_embedding(text, model_name=embedding_model_name):
    """Generates embeddings for a given text using Google's model."""
    try:
        result = ogenai.embed_content(model=f"{model_name}", content=text, task_type="RETRIEVAL_DOCUMENT")
        return result['embedding']
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None


## 3. Initialize Vector Store (ChromaDB - Local)

In [14]:

client = chromadb.PersistentClient(path="./chroma_pv_db") # Creates directory if not exists

# Using Google Generative AI embeddings with ChromaDB
# Note: If using google_ef helper, pass it here. Otherwise, generate embeddings separately.
try:
    # Collection for Literature Abstracts/Snippets
    literature_collection = client.get_or_create_collection(
        name="literature_pv",
        embedding_function=google_ef # Use the helper function
        # metadata={"hnsw:space": "cosine"} # Optional: Specify distance metric
    )
    print(f"ChromaDB collection 'literature_pv' ready. Item count: {literature_collection.count()}")

    # Collection for News/Social Media Posts (potentially shorter text)
    feeds_collection = client.get_or_create_collection(
        name="feeds_pv",
        embedding_function=google_ef
    )
    print(f"ChromaDB collection 'feeds_pv' ready. Item count: {feeds_collection.count()}")

except Exception as e:
    print(f"Error initializing ChromaDB: {e}")

ChromaDB collection 'literature_pv' ready. Item count: 0
ChromaDB collection 'feeds_pv' ready. Item count: 0


### Helper function to add to ChromaDB

In [15]:
import traceback

In [16]:
def add_to_vector_store(collection, documents, metadatas, ids):
    """Adds documents and metadata to the specified ChromaDB collection."""
    if not documents:
        print("No documents to add.")
        return
    try:
        collection.add(
            # embeddings=embeddings, # Not needed if embedding_function is set
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print(f"Added {len(documents)} items to collection '{collection.name}'.")
    except Exception as e:
        print(f"Error adding to ChromaDB collection '{collection.name}': {e}")
        print(traceback.format_exc())



### Helper function to search ChromaDB

In [17]:
def search_vector_store(collection, query_text, n_results=5):
    """Searches the collection for text similar to the query text."""
    try:
        results = collection.query(
            query_texts=[query_text],
            n_results=n_results,
            include=['documents', 'metadatas', 'distances']
        )
        return results
    except Exception as e:
        print(f"Error searching ChromaDB collection '{collection.name}': {e}")
        return None

## 4. Define Knowledge Store (Simulated)
### In a real system, this could be a database or more structured files.

In [18]:
knowledge_store = {
    "drugs": {
        "DrugA": {"class": "ClassX", "indication": "Indication Y", "known_adrs": ["Headache", "Nausea"]},
        "DrugB": {"class": "ClassZ", "indication": "Indication W", "known_adrs": ["Dizziness", "Fatigue", "Rash"]},
    },
    "meddra_mapping": { # Highly simplified mapping example
        "feeling sick": "Nausea",
        "stomach ache": "Abdominal Pain",
        "spots": "Rash",
        "spinning room": "Vertigo",
        "tired": "Fatigue",
        "head hurts": "Headache"
    },
    "seriousness_keywords": ["hospitalized", "disability", "life-threatening", "death", "intervention required", "Fatigue"]
}

In [19]:
def get_drug_info(drug_name):
    return knowledge_store["drugs"].get(drug_name)

In [20]:
def map_to_meddra(term):
    # Simple keyword matching - real system needs fuzzy matching / NLP model
    term_lower = term.lower()
    for key, value in knowledge_store["meddra_mapping"].items():
        if key in term_lower:
            return value
    return term # Return original if no simple map found

In [21]:
def check_seriousness(text):
    text_lower = text.lower()
    for keyword in knowledge_store["seriousness_keywords"]:
        if keyword in text_lower:
            return True
    return False

### --- Agent Function Definitions ---

In [22]:

def literature_scanner_agent(drugs_of_interest, search_terms, max_results=10):
    """
    Monitors PubMed (simulated here) for new relevant literature.
    Extracts potential ADRs and drug mentions.
    Adds findings to vector store.
    Returns list of findings (e.g., AnalyzedItem objects or dicts).
    """
    print(f"\n--- Running Literature Scanner Agent ({datetime.datetime.now()}) ---")
    findings = []
    # TODO: Implement actual PubMed API call using Entrez
    # Entrez.email = "Your.Name.Here@example.org" # Always tell NCBI who you are
    # handle = Entrez.esearch(db="pubmed", term="YourComplexSearchQuery", retmax=max_results)
    # record = Entrez.read(handle)
    # handle.close()
    # etc... fetch abstracts

    # --- Simulation ---
    simulated_abstracts = [
        {"id": "pmid1", "text": "A study on DrugA found increased reports of severe skin reactions, previously unknown.", "drug": "DrugA", "adr": "severe skin reactions"},
        {"id": "pmid2", "text": "DrugB effectiveness was confirmed, common side effects like Fatigue were noted.", "drug": "DrugB", "adr": "Fatigue"},
        {"id": "pmid3", "text": "Interesting case report linking DrugA to sudden onset Vertigo.", "drug": "DrugA", "adr": "Vertigo"},
    ]
    print(f"Simulating PubMed search, found {len(simulated_abstracts)} abstracts.")

    docs_to_embed = []
    metadatas = []
    ids = []

    for abstract in simulated_abstracts:
         # Basic check if drug is relevant
        if abstract["drug"] in drugs_of_interest:
            finding = {
                "source": "pubmed",
                "source_id": abstract["id"],
                "text": abstract["text"],
                "potential_adr": abstract["adr"],
                "drug_mentioned": abstract["drug"],
                "timestamp": datetime.datetime.now()
            }
            findings.append(finding)
            print(f"  Found relevant abstract: {abstract['id']}")

            # Prepare for vector store
            docs_to_embed.append(abstract["text"])
            metadatas.append({
                "source": "pubmed",
                "source_id": abstract["id"],
                "drug": abstract["drug"],
                "adr_mention": abstract["adr"],
                "timestamp": finding["timestamp"].isoformat()
            })
            # Generate a unique, deterministic ID based on content or source ID
            hash_id = hashlib.sha256(abstract["id"].encode()).hexdigest()
            ids.append(f"pubmed_{hash_id}")

    embedded_docs=get_embedding(docs_to_embed)
    # Add findings to Vector Store
    if docs_to_embed:
        add_to_vector_store(literature_collection, documents=docs_to_embed, metadatas=metadatas, ids=ids)

    return findings


In [23]:
def news_listener_agent(drugs_of_interest, keywords, max_results=20):
    """
    Monitors NewsAPI (or RSS) for relevant articles.
    Uses LLM to extract potential ADRs and drug mentions.
    Adds findings to vector store.
    Returns list of findings.
    """
    try:
        print(f"\n--- Running News Listener Agent ({datetime.datetime.now()}) ---")
        findings = []
        # --- Actual NewsAPI Call ---
        # url = f"https://newsapi.org/v2/everything?q={'+OR+'.join(keywords)}&apiKey={NEWS_API_KEY}&pageSize={max_results}&sortBy=publishedAt"
        # try:
        #     response = requests.get(url)
        #     response.raise_for_status() # Raise HTTPError for bad responses (4XX, 5XX)
        #     articles = response.json().get('articles', [])
        # except requests.exceptions.RequestException as e:
        #     print(f"Error fetching news: {e}")
        #     articles = []
    
        # --- Simulation ---
        articles = [
            {"source": {"name": "HealthNews"}, "title": "Concerns grow over DrugA side effects", "description": "Patients report unexpected issues like Vertigo after taking DrugA.", "url": "http://example.com/news1", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Tech Chronicle"}, "title": "New AI for Drug Discovery", "description": "Mentions DrugC development.", "url": "http://example.com/news2", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Med Journal"}, "title": "DrugB trial results positive", "description": "Standard side effects noted, effectiveness confirmed.", "url": "http://example.com/news3", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Med Journal"}, "title": "DrugB trial serious outcome", "description": "Patient reported fatigue, experienced spinning room patient hospitalized", "url": "http://simulated.thiru.com/news4", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Med Journal"}, "title": "DrugB trial serious outcome", "description": "Patient reported spinning room, experienced spinning room patient hospitalized", "url": "http://simulated1.thiru.com/news1", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Med Journal"}, "title": "DrugB trial serious outcome", "description": "Patient reported spinning room, experienced  patient hospitalized", "url": "http://simulated2.thiru.com/news4", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Med Journal"}, "title": "DrugB trial serious outcome", "description": "Patient reported spinning room, experienced spinning room patient hospitalized", "url": "http://simulated3.thiru.com/news5", "publishedAt": datetime.datetime.now().isoformat()},
            {"source": {"name": "Med Journal"}, "title": "DrugB trial serious outcome", "description": "Patient reported spinning room, experienced spinning room patient hospitalized", "url": "http://simulated4.thiru.com/news6", "publishedAt": datetime.datetime.now().isoformat()},
        
        ]
        print(f"Simulating News search, found {len(articles)} articles.")
    
        docs_to_embed = []
        metadatas = []
        ids = []
    
        for article in articles:
            content_to_analyze = f"Title: {article.get('title', '')}\nDescription: {article.get('description', '')}"
    
            # Use LLM to check relevance and extract info
            prompt = f"""Analyze the following news snippet. Does it mention any specific drugs from the list [{', '.join(drugs_of_interest)}]? Does it mention any potential adverse drug reactions or side effects?
    
            News Snippet:
            "{content_to_analyze}"
    
            If it mentions both a relevant drug AND a potential side effect, respond in JSON format with keys "relevant": true, "drug_mentioned": ["list of drugs"], "potential_adr": ["list of adrs"].
            Otherwise, respond with "relevant": false.
            """
            llm_response = call_gemini(prompt, is_json_output=True)
    
            if llm_response and llm_response.get("relevant"):
                drug = llm_response.get("drug_mentioned", [None])[0] # Take first mentioned relevant drug
                adr = llm_response.get("potential_adr", [None])[0] # Take first mentioned relevant adr
    
                if drug in drugs_of_interest and adr:
                    finding = {
                        "source": "news",
                        "source_id": article.get('url', f"news_{hashlib.sha256(content_to_analyze.encode()).hexdigest()}"),
                        "text": content_to_analyze,
                        "potential_adr": adr,
                        "drug_mentioned": drug,
                        "timestamp": datetime.datetime.fromisoformat(article.get('publishedAt').replace("Z", "+00:00")) if article.get('publishedAt') else datetime.datetime.now()
                    }
                    findings.append(finding)
                    print(f"  Relevant news item found: {article.get('url')}")
    
                    # Prepare for vector store
                    docs_to_embed.append(content_to_analyze)
                    metadatas.append({
                        "source": "news",
                        "source_id": finding["source_id"],
                        "drug": drug,
                        "adr_mention": adr,
                        "timestamp": finding["timestamp"].isoformat()
                    })
                    ids.append(f"news_{hashlib.sha256(finding['source_id'].encode()).hexdigest()}")
    
    
        # Add findings to Vector Store
        if docs_to_embed:
            add_to_vector_store(feeds_collection, documents=docs_to_embed, metadatas=metadatas, ids=ids)
    
        return findings
    except Exception as e:
        print(traceback.format_exc())
        return None


### TODO: Add similar agents for Social Media (Reddit/PRAW), simulated FAERS data

In [24]:
def clinical_context_agent(items):
    """
    Analyzes findings from other agents.
    Standardizes ADR terms using Knowledge Store (MedDRA map).
    Checks if ADR is known for the drug.
    Assesses potential seriousness.
    Returns list of contextualized findings.
    """
    print(f"\n--- Running Clinical Context Agent ({datetime.datetime.now()}) ---")
    contextualized_findings = []
    for item in items:
        drug = item['drug_mentioned']
        adr_raw = item['potential_adr']

        # Standardize ADR term
        adr_standardized = map_to_meddra(adr_raw)

        # Check if known ADR for this drug
        drug_info = get_drug_info(drug)
        is_known_adr = False
        if drug_info:
            # Basic check - real system might need fuzzy matching
            is_known_adr = any(known.lower() in adr_standardized.lower() for known in drug_info['known_adrs'])

        # Check seriousness
        is_serious = check_seriousness(item['text']) or check_seriousness(adr_raw)

        # Add context to the finding
        item['adr_standardized'] = adr_standardized
        item['is_known_adr'] = is_known_adr
        item['is_serious'] = is_serious
        contextualized_findings.append(item)
        print(f"  Contextualized: {item['source_id']} - ADR: {adr_standardized} (Known: {is_known_adr}, Serious: {is_serious})")

    return contextualized_findings


In [25]:
def signal_synthesizer_agent(contextualized_items, time_window_days=7, min_reports_for_signal=3):
    """
    Looks for patterns and correlations in contextualized findings.
    Flags potential signals based on criteria (e.g., multiple reports of unexpected ADR).
    Uses Vector Store to find similar past reports.
    Returns list of potential signals (e.g., Signal objects or dicts).
    """
    print(f"\n--- Running Signal Synthesizer Agent ({datetime.datetime.now()}) ---")
    potential_signals = []
    cutoff_date = datetime.datetime.now() - datetime.timedelta(days=time_window_days)

    # Group findings by Drug and Standardized ADR within the time window
    adr_groups = {}
    for item in contextualized_items:
        # Ensure timestamp is timezone-aware or convert naive to aware for comparison
        item_ts = item['timestamp']
        if item_ts.tzinfo is None:
             # Assuming UTC if naive, adjust as needed based on source data timezone
             item_ts = item_ts.replace(tzinfo=datetime.timezone.utc)

        if item_ts < cutoff_date.replace(tzinfo=datetime.timezone.utc): # Make cutoff aware too
            continue

        key = (item['drug_mentioned'], item['adr_standardized'])
        if key not in adr_groups:
            adr_groups[key] = []
        adr_groups[key].append(item)

    # Analyze groups
    for (drug, adr), items in adr_groups.items():
        # Basic Signal Criteria: Multiple reports of an *unexpected* and potentially *serious* ADR
        num_reports = len(items)
        is_unexpected = not items[0]['is_known_adr'] # Assumes consistency within group
        has_serious_report = any(item['is_serious'] for item in items)

        # Example Rule: Signal if >= min_reports of an unexpected ADR, OR if >= N reports of a serious ADR (even if known)
        if (is_unexpected and num_reports >= min_reports_for_signal) or \
           (has_serious_report and num_reports >= min_reports_for_signal + 2): # Stricter threshold for serious

            # Use Vector Store to find related historical items (optional enhancement)
            # query = f"Reports related to {drug} and {adr}"
            # similar_historical_reports = search_vector_store(literature_collection, query, n_results=5)
            # print(f"Found similar historical reports: {similar_historical_reports}")

            # Generate Signal
            evidence_summary = f"Found {num_reports} reports of '{adr}' for {drug} within the last {time_window_days} days. "
            evidence_summary += f"This ADR is considered {'unexpected' if is_unexpected else 'known'}. "
            if has_serious_report:
                evidence_summary += "At least one report mentioned serious outcomes. "
            source_ids = [item['source_id'] for item in items]

            signal = {
                "drugs": [drug],
                "adr_term": adr,
                "evidence": evidence_summary,
                "sources": source_ids,
                "confidence_score": 0.6 + min(0.4, (num_reports / (min_reports_for_signal * 2))), # Simple confidence heuristic
                "timestamp": datetime.datetime.now()
            }
            potential_signals.append(signal)
            print(f"  Potential Signal Identified: {drug} - {adr}")
            print(f"    Evidence: {evidence_summary}")

    return potential_signals



### 6. Main Workflow Orchestration


In [26]:
DRUGS_TO_MONITOR = ["DrugA", "DrugB"] # Example list
KEYWORDS_FOR_NEWS = DRUGS_TO_MONITOR + ["side effect", "adverse reaction"] # Example keywords
KEYWORDS_FOR_LITERATURE = [f"{drug}[Title/Abstract] AND (adverse event[MeSH Terms] OR side effect[Title/Abstract])" for drug in DRUGS_TO_MONITOR] # Example PubMed query parts

### --- Run Agents Sequentially (Simple Orchestration) ---
### In a real system, this could run on a schedule (e.g., daily, hourly)

#### 1. Data Ingestion / Scanning

In [27]:
lit_findings = literature_scanner_agent(DRUGS_TO_MONITOR, KEYWORDS_FOR_LITERATURE)
# news_findings = news_listener_agent(DRUGS_TO_MONITOR, KEYWORDS_FOR_NEWS)
# social_findings = social_listener_agent(...) # TODO
# faers_findings = faers_processor_agent(...) # TODO
all_raw_findings = lit_findings #+ news_findings # + social_findings + faers_findings


--- Running Literature Scanner Agent (2025-04-17 16:33:56.829447) ---
Simulating PubMed search, found 3 abstracts.
  Found relevant abstract: pmid1
  Found relevant abstract: pmid2
  Found relevant abstract: pmid3
Added 3 items to collection 'literature_pv'.


In [28]:
news_findings = news_listener_agent(DRUGS_TO_MONITOR, KEYWORDS_FOR_NEWS)


--- Running News Listener Agent (2025-04-17 16:33:57.798336) ---
Simulating News search, found 8 articles.
  Relevant news item found: http://example.com/news1
  Relevant news item found: http://example.com/news3
  Relevant news item found: http://simulated.thiru.com/news4
  Relevant news item found: http://simulated1.thiru.com/news1
  Relevant news item found: http://simulated2.thiru.com/news4
  Relevant news item found: http://simulated3.thiru.com/news5
  Relevant news item found: http://simulated4.thiru.com/news6
Added 7 items to collection 'feeds_pv'.


In [29]:
all_raw_findings = lit_findings + news_findings

In [30]:
print(all_raw_findings)

[{'source': 'pubmed', 'source_id': 'pmid1', 'text': 'A study on DrugA found increased reports of severe skin reactions, previously unknown.', 'potential_adr': 'severe skin reactions', 'drug_mentioned': 'DrugA', 'timestamp': datetime.datetime(2025, 4, 17, 16, 33, 56, 829527)}, {'source': 'pubmed', 'source_id': 'pmid2', 'text': 'DrugB effectiveness was confirmed, common side effects like Fatigue were noted.', 'potential_adr': 'Fatigue', 'drug_mentioned': 'DrugB', 'timestamp': datetime.datetime(2025, 4, 17, 16, 33, 56, 830751)}, {'source': 'pubmed', 'source_id': 'pmid3', 'text': 'Interesting case report linking DrugA to sudden onset Vertigo.', 'potential_adr': 'Vertigo', 'drug_mentioned': 'DrugA', 'timestamp': datetime.datetime(2025, 4, 17, 16, 33, 56, 830781)}, {'source': 'news', 'source_id': 'http://example.com/news1', 'text': 'Title: Concerns grow over DrugA side effects\nDescription: Patients report unexpected issues like Vertigo after taking DrugA.', 'potential_adr': 'Vertigo', 'drug

### TODO: Add similar agents for Social Media (Reddit/PRAW), simulated FAERS data


In [31]:
def clinical_context_agent(items):
    """
    Analyzes findings from other agents.
    Standardizes ADR terms using Knowledge Store (MedDRA map).
    Checks if ADR is known for the drug.
    Assesses potential seriousness.
    Returns list of contextualized findings.
    """
    print(f"\n--- Running Clinical Context Agent ({datetime.datetime.now()}) ---")
    contextualized_findings = []
    for item in items:
        drug = item['drug_mentioned']
        adr_raw = item['potential_adr']

        # Standardize ADR term
        adr_standardized = map_to_meddra(adr_raw)

        # Check if known ADR for this drug
        drug_info = get_drug_info(drug)
        is_known_adr = False
        if drug_info:
            # Basic check - real system might need fuzzy matching
            # print(f"Checking {drug} and {adr_raw} in {adr_standardized} and {drug_info['known_adrs']}")
            is_known_adr = any(known.lower() in adr_standardized.lower() for known in drug_info['known_adrs'])

        # Check seriousness
        is_serious = check_seriousness(item['text']) or check_seriousness(adr_raw)

        # Add context to the finding
        item['adr_standardized'] = adr_standardized
        item['is_known_adr'] = is_known_adr
        item['is_serious'] = is_serious
        contextualized_findings.append(item)
        print(f"  Contextualized: {item['source_id']} - ADR: {adr_standardized} (Known: {is_known_adr}, Serious: {is_serious})")

    return contextualized_findings

In [32]:
def signal_synthesizer_agent(contextualized_items, time_window_days=7, min_reports_for_signal=3):
    """
    Looks for patterns and correlations in contextualized findings.
    Flags potential signals based on criteria (e.g., multiple reports of unexpected ADR).
    Uses Vector Store to find similar past reports.
    Returns list of potential signals (e.g., Signal objects or dicts).
    """
    print(f"\n--- Running Signal Synthesizer Agent ({datetime.datetime.now()}) ---")
    potential_signals = []
    cutoff_date = datetime.datetime.now() - datetime.timedelta(days=time_window_days)

    # Group findings by Drug and Standardized ADR within the time window
    adr_groups = {}
    for item in contextualized_items:
        # Ensure timestamp is timezone-aware or convert naive to aware for comparison
        item_ts = item['timestamp']
        if item_ts.tzinfo is None:
             # Assuming UTC if naive, adjust as needed based on source data timezone
             item_ts = item_ts.replace(tzinfo=datetime.timezone.utc)

        if item_ts < cutoff_date.replace(tzinfo=datetime.timezone.utc): # Make cutoff aware too
            continue

        key = (item['drug_mentioned'], item['adr_standardized'])
        if key not in adr_groups:
            adr_groups[key] = []
        adr_groups[key].append(item)

    # Analyze groups
    for (drug, adr), items in adr_groups.items():
        # Basic Signal Criteria: Multiple reports of an *unexpected* and potentially *serious* ADR
        num_reports = len(items)
        is_unexpected = not items[0]['is_known_adr'] # Assumes consistency within group
        has_serious_report = any(item['is_serious'] for item in items)

        # Example Rule: Signal if >= min_reports of an unexpected ADR, OR if >= N reports of a serious ADR (even if known)
        if (is_unexpected and num_reports >= min_reports_for_signal) or \
           (has_serious_report and num_reports >= min_reports_for_signal + 2): # Stricter threshold for serious

            # Use Vector Store to find related historical items (optional enhancement)
            # query = f"Reports related to {drug} and {adr}"
            # similar_historical_reports = search_vector_store(literature_collection, query, n_results=5)
            # print(f"Found similar historical reports: {similar_historical_reports}")

            # Generate Signal
            evidence_summary = f"Found {num_reports} reports of '{adr}' for {drug} within the last {time_window_days} days. "
            evidence_summary += f"This ADR is considered {'unexpected' if is_unexpected else 'known'}. "
            if has_serious_report:
                evidence_summary += "At least one report mentioned serious outcomes. "
            source_ids = [item['source_id'] for item in items]

            signal = {
                "drugs": [drug],
                "adr_term": adr,
                "evidence": evidence_summary,
                "sources": source_ids,
                "confidence_score": 0.6 + min(0.4, (num_reports / (min_reports_for_signal * 2))), # Simple confidence heuristic
                "timestamp": datetime.datetime.now()
            }
            potential_signals.append(signal)
            print(f"  Potential Signal Identified: {drug} - {adr}")
            print(f"    Evidence: {evidence_summary}")

    return potential_signals

### 6. Main Workflow Orchestration

In [33]:
DRUGS_TO_MONITOR = ["DrugA", "DrugB"] # Example list
KEYWORDS_FOR_NEWS = DRUGS_TO_MONITOR + ["side effect", "adverse reaction"] # Example keywords
KEYWORDS_FOR_LITERATURE = [f"{drug}[Title/Abstract] AND (adverse event[MeSH Terms] OR side effect[Title/Abstract])" for drug in DRUGS_TO_MONITOR] # Example PubMed query parts

### --- Run Agents Sequentially (Simple Orchestration) ---
### In a real system, this could run on a schedule (e.g., daily, hourly)

### 1. Data Ingestion / Scanning

In [34]:
lit_findings = literature_scanner_agent(DRUGS_TO_MONITOR, KEYWORDS_FOR_LITERATURE)
news_findings = news_listener_agent(DRUGS_TO_MONITOR, KEYWORDS_FOR_NEWS)
# social_findings = social_listener_agent(...) # TODO
# faers_findings = faers_processor_agent(...) # TODO

all_raw_findings = lit_findings + news_findings # + social_findings + faers_findings


--- Running Literature Scanner Agent (2025-04-17 16:34:05.373607) ---
Simulating PubMed search, found 3 abstracts.
  Found relevant abstract: pmid1
  Found relevant abstract: pmid2
  Found relevant abstract: pmid3
Added 3 items to collection 'literature_pv'.

--- Running News Listener Agent (2025-04-17 16:34:06.130341) ---
Simulating News search, found 8 articles.
  Relevant news item found: http://example.com/news1
  Relevant news item found: http://example.com/news3
  Relevant news item found: http://simulated.thiru.com/news4
  Relevant news item found: http://simulated1.thiru.com/news1
  Relevant news item found: http://simulated2.thiru.com/news4
  Relevant news item found: http://simulated3.thiru.com/news5
  Relevant news item found: http://simulated4.thiru.com/news6
Added 7 items to collection 'feeds_pv'.


In [35]:
# 2. Contextualization
contextualized_findings = clinical_context_agent(all_raw_findings)


--- Running Clinical Context Agent (2025-04-17 16:34:12.943588) ---
  Contextualized: pmid1 - ADR: severe skin reactions (Known: False, Serious: False)
  Contextualized: pmid2 - ADR: Fatigue (Known: True, Serious: False)
  Contextualized: pmid3 - ADR: Vertigo (Known: False, Serious: False)
  Contextualized: http://example.com/news1 - ADR: Vertigo (Known: False, Serious: False)
  Contextualized: http://example.com/news3 - ADR: Standard side effects (Known: False, Serious: False)
  Contextualized: http://simulated.thiru.com/news4 - ADR: fatigue (Known: True, Serious: True)
  Contextualized: http://simulated1.thiru.com/news1 - ADR: Vertigo (Known: False, Serious: True)
  Contextualized: http://simulated2.thiru.com/news4 - ADR: Vertigo (Known: False, Serious: True)
  Contextualized: http://simulated3.thiru.com/news5 - ADR: Vertigo (Known: False, Serious: True)
  Contextualized: http://simulated4.thiru.com/news6 - ADR: Vertigo (Known: False, Serious: True)


In [36]:
# 3. Synthesis / Signal Detection
detected_signals = signal_synthesizer_agent(contextualized_findings)


--- Running Signal Synthesizer Agent (2025-04-17 16:34:12.985031) ---
  Potential Signal Identified: DrugB - Vertigo
    Evidence: Found 4 reports of 'Vertigo' for DrugB within the last 7 days. This ADR is considered unexpected. At least one report mentioned serious outcomes. 


In [37]:
# # 7. Output / Demonstration

print("\n\n--- Final Detected Signals ---")
if detected_signals:
    for i, signal in enumerate(detected_signals):
        print(f"\nSignal {i+1}:")
        print(f"  Drug(s): {', '.join(signal['drugs'])}")
        print(f"  ADR Term: {signal['adr_term']}")
        print(f"  Evidence: {signal['evidence']}")
        print(f"  Confidence: {signal['confidence_score']:.2f}")
        print(f"  Supporting Sources: {', '.join(signal['sources'])}")
        print(f"  Timestamp: {signal['timestamp']}")
else:
    print("No significant signals detected in this run.")



--- Final Detected Signals ---

Signal 1:
  Drug(s): DrugB
  ADR Term: Vertigo
  Evidence: Found 4 reports of 'Vertigo' for DrugB within the last 7 days. This ADR is considered unexpected. At least one report mentioned serious outcomes. 
  Confidence: 1.00
  Supporting Sources: http://simulated1.thiru.com/news1, http://simulated2.thiru.com/news4, http://simulated3.thiru.com/news5, http://simulated4.thiru.com/news6
  Timestamp: 2025-04-17 16:34:12.985176


In [38]:
get_drug_info("DrugB")

{'class': 'ClassZ',
 'indication': 'Indication W',
 'known_adrs': ['Dizziness', 'Fatigue', 'Rash']}